# This Function will reformat and analyse market data from RAVs in order to dimension the Edge Cloud Hub

In [1]:
# Careful this is the third version
def create_clusters(lsee_in, cluster_max,see):
    # Initialize variables and set up first comparison
    lsee = lsee_in.copy()
    # We will use this variable to count towards cluster max size
    count = 0
    # Use low_see to turn off printing in the sub-routines, but leave it on here
    low_see = False
    # Use this variable to record the eNB id of the cluster
    clusterenb = lsee.get_value(0,'Genid')
    start = int(clusterenb)
    if see:
        print 'Starting clustereNb ', start
    if not cluster_max:
        cluster_max = 100
    if see:
        print cluster_max
        # Loop to assign cluster keys every cluster_max groups unless they are the same location
    row_prev_ = get_lon(lsee,0,low_see)
    row_prev = float(row_prev_[1])
    ##################################################

    df_size = len(lsee.index)
    
    for i in range(df_size):
        count +=1
        row_curr_ = get_lon(lsee,i,low_see)
        row_curr = float(row_curr_[1])
        #rowv_,sindex = get_enb(lsee,i,low_see)
        #rowv = int(rowv_[0])
        clusterenb = lsee.get_value(i,'clustereNb')
        ##################################################
        # We could get the distance of the current row to the proposed cluster 
        # If the distance is beyond a limit, we could force a new cluster, this is overkill since
        # others will have a k-means clustering algorithm.
        ##################################################
        # Check 4 cases 
        if row_curr == row_prev and count <cluster_max:
            #add to cluster by taking genid and setting to clusterenb;
            lsee.set_value(i,'clustereNb', start)

            if low_see:
                print "Case 1- Same Site Add to Group \n", 
        elif row_curr == row_prev and count >=cluster_max:
            # add to cluster pointed to by start
            lsee.set_value(i,'clustereNb', start)
            
            if low_see:
                print "Case 2- OverFlow \n"
        elif row_curr != row_prev and count <cluster_max:
            # add to cluster pointed to by start
            lsee.set_value(i,'clustereNb', start)
            
            if low_see:
                print "**************Case 3- New site Add to Group \n", 
        elif row_curr != row_prev and count >=cluster_max:
            #create new cluster; add self; reset count and reset start pointer
            clusterenb = lsee.get_value(i,'Genid')
            start = int(clusterenb)
            if low_see:
                print "start ", start, 'count ', count

            count = 0
            lsee.set_value(i,'clustereNb', start)
            if low_see:
                print "start ", start, 'count ', count
                print "**************Case 4- New Group \n",
            
        else:
            if True:
                print 'Case 5***************'
                print 'This should never be true'
                print 'row_curr ', row_curr, 'row_prev ', row_prev, 'count ', count, 'i ', i
        row_prev = row_curr
    if low_see:
        print 'Test cluster to use ', clusterenb, start, i
    return (lsee)


In [2]:
# simply pythonic way to set the clusereNb id  by eNBid

def get_genid(grp):
    value = grp.get_value(0,0,'Genid')
    grp['clustereNb']= int(value)
    return grp
#lsee is the main dataframe now with the updated cluster eNb ids

In [3]:
def get_df(inputFile):
    import os
    import pandas as pd
    import csv
    # A Place to override the passed filename
    filename = 'RM_MSA_LookUP.csv'
    try:
        with open(inputFile) as mapfile:
            mapfile_df = pd.read_csv(mapfile,skipinitialspace=True,dtype=unicode)
    except:
            print "something went wrong"
    return mapfile_df

In [4]:
def clusterdist(lsee2,see):
    #  2nd Attempt More pythonic way
    see = False
    low_see = False
    import calc_distance
    df_size = len(lsee2.index)
    distance = 0.0
    count = 0

    print "Size of List ", (df_size)

    for row in range(df_size):
        clusterenb = lsee2.get_value(row,'clustereNb')
        lat1 = lsee2.get_value(row,'Lat')
        lon1 = lsee2.get_value(row,'Lon')
        # Need to do a look up to find the clustereNb coordinates
        # Find where pip is in the original dataframe  an index, there may be multiple instance take the first
        index_list = lsee2.loc[lsee2['Genid'] ==clusterenb].index.tolist()
        tcluster = int(index_list[0])

        tlat2 = lsee2.loc[tcluster,['Lat']].values.tolist()
        tlon2 = lsee2.loc[tcluster,['Lon']].values.tolist()
        lat2 = float(tlat2[0])
        lon2 = float(tlon2[0])
        # Call the distance function
        distance = calc_distance.get_dist_to_cell(lat1,lon1,lat2,lon2)
        lsee2.set_value(row,'clusterdist',distance)

        if low_see:
            print 'row', row,'Distance in Km', distance,
            print 'lat1 ','lon1 ', lat1, lon1,  'lat2 ', 'lon2 ', lat2, lon2  
        if distance > 5.0:
            count +=1
            if see:
                print 'Distance in Km', distance,'clusterenb ', clusterenb," " , row
    print 'sites > 5km = ', count
    return (lsee2)

In [5]:
#Functions to get values from the dataframes based on index
def get_lon (df,test,see):

    # Step 0 of 3
    value_list = df[['Genid','Lon']].values.tolist()
    # Ste 1 of 3 find the index to the first value 2nd column 
    value = value_list[test][1]
    if see:
        print 'get_lon: value ', value
    # still in step 1:
    lon_index = df.loc[df['Lon']==value].index.tolist()
    if see:
        print 'get_lon: lon_index ', lon_index

    # Step 2 of 3
    sindex = lon_index[0]
    if see:
        print 'get_lon: sindex', sindex
    # Step 3A of 3
    row_prev = df.loc[sindex,['Genid','Lon']].values.tolist()
    if see:
        print 'get_lon:  row_prev ', row_prev
    return row_prev
# Repeat to get the eNB id
###################################################
def get_enb (df,test,see):

    # Step 0 of 3
    # Aleady computed above no need to redo
    value_list = df[['Genid','Lon']].values.tolist()
    # Step 1 of 3 find the index to the first value 2nd column 
    value = value_list[test][0]
    if see:
        print 'get_enb: value ', value
    # still in step 1:
    geid_index = df.loc[df['Genid']==value].index.tolist()
    if see:
        print 'get_enb: lon_index ', geid_index

    # Step 2 of 3
    sindex = geid_index[0]
    if see:
        print 'get_enb: sindex', sindex
    # Step 3A of 3
    rowv = df.loc[sindex,['Genid']].values.tolist()
    if see:
        print 'get_enb:  rowv ', rowv
    # let us try modifying a value
    if see:
        print df.get_value(sindex,'clustereNb')
    return rowv, sindex

#Writing to Dataframe

#df.set_value(sindex,'clustereNb', 81414)
#print 'New value ',df.get_value(sindex,'clustereNb')

First we will read in the data using Pandas Data Frames from a raw UTF-8 *.csv file.

In [6]:
import pandas as pd
import numpy as np
import pprint

#Globally enable debug printing
see = False
#%matplotlib notebook
inputFile = "cellsummaryshort.csv"
inputFile = "cellsummaryNYM.csv"
cluster_max = int(256)

df = get_df(inputFile)

In [7]:
# Read in a list of Columns.
list(df)
pp = pprint.PrettyPrinter(depth=4, width =10)
if see:
    pp.pprint(list(df))

In [8]:
#rename some very long column names
df.rename(columns={'Number of Downlink Antennas':'Tx'}, inplace=True)
df.rename(columns={'Number of Uplink Antennas':'Rx'}, inplace=True)
# Downlink Bandwidth to DLBW
df.rename(columns={ 'Downlink Bandwidth':'DLBW'}, inplace=True)
# Cell Latitude and Cell Longitude to Lat Lon
df.rename(columns={ 'Cell Latitude':'Lat'}, inplace=True)
df.rename(columns={ 'Cell Longitude':'Lon'}, inplace=True)

# get rid of spaces!!
df.rename(columns={ 'Cell Id':'Cellid'}, inplace=True)
df.rename(columns={ 'Global eNodeB ID':'Genid'}, inplace=True)
df.rename(columns={ 'LTE Cell Name':'lteCellid'}, inplace=True)



#clean up Uplink Antennas and Downlink Bandwidth columns to be a number and more readable column
df.replace('ulAntenna1',1, inplace=True)
df.replace('ulAntenna2',2, inplace=True)
df.replace('ulAntenna4',4, inplace=True)
# We could transform these strings to numbers in the future, for now just identifying them is sufficient
df.replace('n75-15MHz', '15MHz', inplace=True)
df.replace('n50-10MHz', '10MHz', inplace=True)
df.replace('n100-20MHz', '20MHz', inplace=True)


# Compute the total number of unique eNBs

count = df['Genid'].groupby(df['Genid']).unique()
#count = df['Genid'].unique()
print "Number of eNBs",count.size


#this may not be returning the correct values, the number of unique sites can not be 
# Greater than the total number eNBs!
#countsite = pd.concat([df['Lat'],df['Lon']]).unique()
countsite = np.unique(df[['Lat']])
print "Number of sites", countsite.size
max_enb =count.size

# Compute the total number of unique Cells

enb_df = pd.value_counts(df['Cellid'].values, sort=True)
count = enb_df.values
print ' Number of Cells = ',count.sum()

Number of eNBs 3638
Number of sites 3038
 Number of Cells =  18780


# Sort the data by lat and longitude.

In [9]:
# Sort Rows by lat, long
# Create a list
cell_sort = []
cell_sort = [ 'Lat', 'Lon']
# Not used
#df_loc_sort = df.sort(cell_sort)
#df.sort(cell_sort, inplace=True)
# Sorting by both Lat and Lon produce interesting results, so we will just choose Lon
try:
    df.sort(['Lon','Genid'], inplace=True)
    df.reset_index(drop = True, inplace=True)
    df= df.convert_objects(convert_numeric=True)
except:
    print 'There was an error sorting in place and converting objects to numerical values'

if see:
    df['Genid'].groupby(df['Cellid']).describe()

Remove columns in the data that are not relevant to the study

In [10]:
# This might change with a different formatted data set if the columns above change.
ndf = df.drop(
 ['\xef\xbb\xbfNetwork Element','eNB ePC Name',
  'eNodeB Name',
 'eNodeB Alias Name',
 'eNodeB Admin State',
 'PLMN Country Code',
 'PLMN Network Code',
 'Physical Cell Identity',
 'LTE Cell Status',
 'Running SW Version',
 'Tracking Area',
 'Cell Radius',
 'Downlink EARFCN',
 'Uplink EARFCN',
  'Uplink Bandwidth',
 'Downlink power',
 '700 MHz',
 '2x2 MIMO',
 '4x2 MIMO',
  'Azimuth'],axis=1)


Add Columns to represent the cluster eNB and cluster id

In [11]:
# Add clustereNB and clusterdist columns
#take the eNb id column and make a list
eNb = []
# One way to populate the clustereNb is to see it with the actual Global eNb Id
eNb = ndf['Genid'].astype(int).tolist()
# now make that list the initial clustereNb column
ndf['clustereNb'] = pd.Series(eNb, index=ndf.index)
# The preferred way is to use random numbers, but we need integers!!
# Override if uncommended with random values.  Either way is fine.
try:
    # now make that list the initial clustereNb column
    ndf['clustereNb'] = pd.Series(eNb, index=ndf.index)
    ndf['clusterdist'] = pd.Series(np.random.randn(), index=ndf.index)

    #ndf['clustereNb'] = pd.Series(np.random.randn(), index=ndf.index)
    #ndf['clustereNb'] = pd.Series(np.random.randint(88888,high=99999), index=ndf.index)
except:
    print 'Random initialization Failed'

if see:
    print ndf[21:28][['Genid','clustereNb','clusterdist']]
    print ndf[0:5][['Genid','clustereNb']]


This provides us with a beginning list of cluster eNB locations equal to the same site. Below we update that list with first eNB in the group by location. Later we will replace the cluster ids with the actual cluster site locations based on N number of clusters combined together.

In [12]:
# temporarliy turning off assigning eNBid by locations
#lsee= ndf.groupby(['Lon']).apply(get_genid)
# it is import to keep the indexing of the DF correct or the clustering routine will fail
lsee =ndf.copy(deep = True)
lsee.reset_index(drop = False, inplace= True)

Next executions set up variables to create a key for the locations and the cluster eNBs
We will loop through the entire set of eNBs grouping by cluster_max to create our sample clusters of cluster_max size. The loop takes care to avoid splitting locations so in some cases the clusters may be slightly bigger than 100 cells.

In [13]:
# Temp overide over cluster max for testing
import time
see = 0; # Turn off printing in the function
start_time = time.time()
# Control whether to perform the long loop for creating clusters
if True:
    lsee2 = create_clusters(lsee,cluster_max,see)
    #Now generate cluster distances
    lsee2 = clusterdist(lsee2,see)
else:
    lsee2 = lsee.copy()

count = lsee2['clustereNb'].unique()
print count.size, 'Number of Unique Clusters of size ~=  ', cluster_max, time.time()- start_time




Size of List  18780
sites > 5km =  12672
71 Number of Unique Clusters of size ~=   256 208.319000006


In [14]:

if see:
    print ' Resulting Before clustering \n',ndf[['Genid','clustereNb','Lat','Lon']][(ndf['Genid']==78778)]
    print ' Resulting after clustering \n',lsee2[['Genid','clustereNb']][(lsee2['clustereNb']==78778)]

# Write out to a file using the pandas to_excel function

In [15]:
# First let's create a grouping that provides only the necessary data and unstacks the rows for excel
#This grouping provides a count of cells per cluster with informaton on the antenna config + BW.
grouped =lsee2[['Genid','Cellid','Rx','Tx','clustereNb','DLBW','Lat','Lon','clusterdist']].\
groupby(['clustereNb','Genid','DLBW','Rx','Tx','Lat','Lon','clusterdist']).size().reset_index(name='count')

# Python and pandas requires you perform some operation to turn the groupedby DF into a regular DF. 
# So this is a dummy operation
grouped.sum(inplace = False)

# Build the file name to be used based on the number of clusters chosen
filename = 'NYMCluster-' + str(cluster_max)+ 'cell.xls'
try:
    grouped.to_excel(filename, sheet_name='clusters')
    print 'Wrote ', filename, " to directory in excel format"
except:
    print "File writing didn't succeed"

Wrote  NYMCluster-100cell.xls  to directory in excel format
